In [2]:
import pandas as pd
from datetime import datetime

In [9]:
df1 = pd.read_csv('Data/GOES_16_merged_data.csv')
df2 = pd.read_csv('Data/GOES_17_merged_data.csv')
df3 = pd.read_csv('Data/GOES_18_merged_data.csv')

/var/folders/r7/6cts6h556zzgbx26m_g0p4gw0000gn/T/ipykernel_919/2651289391.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Data/GOES_17_merged_data.csv')


In [10]:
df1.head(1000)

,time,xrsa_flux,xrsb_flux,status,flare_class
0,2017-02-07 00:00:00,2.949822e-08,7.072423e-08,NaN,NaN
1,2017-02-07 00:01:00,2.716477e-08,6.869706e-08,NaN,NaN
2,2017-02-07 00:02:00,2.996076e-08,6.933664e-08,NaN,NaN
3,2017-02-07 00:03:00,3.124979e-08,6.796333e-08,NaN,NaN
4,2017-02-07 00:04:00,3.095799e-08,6.928261e-08,NaN,NaN
...,...,...,...,...,...
995,2017-02-07 16:35:00,4.418819e-09,4.118933e-08,NaN,NaN
996,2017-02-07 16:36:00,3.437882e-09,3.999427e-08,NaN,NaN
997,2017-02-07 16:37:00,6.502486e-09,4.040907e-08,NaN,NaN
998,2017-02-07 16:38:00,5.297591e-09,3.889536e-08,NaN,NaN


In [11]:
merged_df = pd.merge(df1, df2, on='time', how='outer')
merged_df = pd.merge(merged_df, df3, on='time', how='outer')
print(len(merged_df))


3548372


In [12]:
merged_df = merged_df[["time","xrsa_flux_x","xrsb_flux_x","status_x","flare_class_x"]]
merged_df = merged_df.rename(columns={"xrsa_flux_x": "xrsa_flux", "xrsb_flux_x": "xrsb_flux", 'status_x':'status', 'flare_class_x': 'flare_class'})

mask = merged_df['xrsa_flux'] != merged_df['xrsa_flux'].isna

merged_df = merged_df[mask]

merged_df = merged_df.drop_duplicates(subset='time', keep='first')
print(len(merged_df))

3548160


In [13]:
print(merged_df["time"].is_unique)

True


In [14]:
duplicates = merged_df[merged_df.duplicated('time', keep=False)]

# Display duplicate rows
print("Duplicate Time Values:")
print(duplicates)
print(len(duplicates))
print(len(merged_df))
has_duplicates = merged_df['time'].duplicated().any()
print(f"Are there duplicate time values? {has_duplicates}")

Duplicate Time Values:
Empty DataFrame
Columns: [time, xrsa_flux, xrsb_flux, status, flare_class]
Index: []
0
3548160
Are there duplicate time values? False


In [15]:
first_value = merged_df.head(1)
first_value_time = str(first_value['time'].item())
first_value_time = datetime.strptime(first_value_time, '%Y-%m-%d %H:%M:%S')
print(first_value_time)

2017-02-07 00:00:00


In [16]:
last_value = merged_df.tail(1)
last_value_time = str(last_value['time'].item())
last_value_time = datetime.strptime(last_value_time, '%Y-%m-%d %H:%M:%S')
print(last_value_time)

2023-11-06 23:59:00


In [17]:
time_difference = last_value_time - first_value_time
amount_of_minutes = int(time_difference.total_seconds() / 60)
print(amount_of_minutes)

3548159


In [18]:
print(len(merged_df))
print()

3548160


In [19]:
merged_df.to_csv("full_set.csv", index=False)

# Transforming the Dataset from long format to wide  Format 

In [72]:
merged_df = pd.read_csv('full_set.csv')

In [73]:
merged_df = merged_df[["time","xrsa_flux","status"]]

In [75]:
in_event = False
event_label = 0
labels = []

for status in merged_df['status']:
    if status == 'EVENT_PEAK':
        event_label = 1
    else:
        event_label = 0
    labels.append(event_label)

cleaned = merged_df[["time","xrsa_flux"]]
cleaned['Label'] = labels

# removing all the 2017 data
cleaned['time'] = pd.to_datetime(cleaned['time'])

start_time = pd.to_datetime('2018-01-01 00:00:00')
cleaned = cleaned[(cleaned['time'] >= start_time)]

cleaned = cleaned.dropna(subset=['xrsa_flux'])
cleaned = cleaned.reset_index(drop=True)

cleaned.head(10000)  

,time,xrsa_flux,Label
0,2018-01-01 00:00:00,5.246326e-09,0
1,2018-01-01 00:01:00,3.435355e-09,0
2,2018-01-02 00:01:00,6.933772e-09,0
3,2018-01-02 00:02:00,5.229108e-09,0
4,2018-01-02 00:03:00,6.279681e-09,0
...,...,...,...
9995,2018-01-09 00:29:00,5.455672e-09,0
9996,2018-01-09 00:30:00,4.537092e-09,0
9997,2018-01-09 00:31:00,5.552804e-09,0
9998,2018-01-09 00:32:00,5.606264e-09,0


In [87]:
df = cleaned

# Even though the window size was about 183 readings, I think it might be too big, Let's start testing out 100
window_size = 184

wide_series_list = []

i = window_size
while i < len(df) - window_size + 1:
    window = df.iloc[i - (window_size//2):i + (window_size//2)]
    if window['Label'].any() == 1:
        middle_index = (window['Label'] == 1).idxmax()
        if middle_index == i:
            window = list(window['xrsa_flux'].reset_index(drop=True))
            window.append(1)
            wide_series_list.append(window)
            i = i + (window_size//2) + 1
            continue
        else: 
            i += 1
            continue
    else:
        window = list(window['xrsa_flux'].reset_index(drop=True))
        window.append(0)
        wide_series_list.append(window)
        i = i + (window_size//2) + 1
        

In [88]:
print(len(wide_series_list))
print(type(wide_series_list[0]))

transposed_df = pd.DataFrame(wide_series_list)

# wide_series_df = pd.concat(wide_series_list, axis=1)
# transposed_df = wide_series_df.T

# print(len(transposed_df))


28707
<class 'list'>


In [89]:
transposed_df.columns = [str(i) for i in range(window_size)] + ['Label'] 
transposed_df.head(2000)

,0,1,2,3,4,5,6,7,8,9,...,175,176,177,178,179,180,181,182,183,Label
0,5.533539e-09,5.533268e-09,6.185512e-09,4.711178e-09,4.654437e-09,4.010655e-09,4.509709e-09,4.850949e-09,3.753756e-09,5.159274e-09,...,3.131046e-09,3.963064e-09,4.339098e-09,2.207876e-09,2.893943e-09,3.379195e-09,1.902864e-09,3.477053e-09,2.947734e-09,0
1,1.748377e-09,3.118675e-09,2.304878e-09,2.037305e-09,2.640252e-09,3.238086e-09,2.848076e-09,3.076982e-09,3.475040e-09,4.013863e-09,...,3.957633e-09,2.427869e-09,3.239848e-09,2.494678e-09,2.997068e-09,4.503659e-09,3.203841e-09,3.156329e-09,2.559061e-09,0
2,3.454376e-09,3.177857e-09,2.937502e-09,3.593601e-09,4.243432e-09,3.359992e-09,4.511997e-09,2.948331e-09,2.693326e-09,2.726117e-09,...,5.742270e-09,5.984768e-09,5.355942e-09,5.724618e-09,4.386251e-09,3.874365e-09,4.410673e-09,5.113217e-09,5.357295e-09,0
3,3.659730e-09,4.170499e-09,3.420095e-09,3.123178e-09,2.959743e-09,2.773871e-09,2.636297e-09,5.295369e-09,2.986771e-09,3.957314e-09,...,3.462123e-09,5.206575e-09,5.104474e-09,4.504015e-09,3.408692e-09,4.891970e-09,4.480620e-09,4.982614e-09,5.536753e-09,0
4,4.697617e-09,4.834471e-09,3.777643e-09,5.720345e-09,4.436036e-09,3.994602e-09,4.199571e-09,4.790939e-09,4.633575e-09,4.326550e-09,...,7.139244e-09,8.846097e-09,6.985636e-09,7.452307e-09,7.490661e-09,7.703980e-09,8.349361e-09,8.429148e-09,7.828551e-09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.718567e-09,4.957889e-09,6.201355e-09,4.045499e-09,5.046086e-09,6.073011e-09,6.042229e-09,6.340288e-09,7.174432e-09,7.168192e-09,...,5.823370e-09,7.084416e-09,7.139876e-09,9.027745e-09,7.203816e-09,7.548300e-09,7.305786e-09,7.777071e-09,6.376212e-09,0
1996,5.743389e-09,6.442411e-09,4.196084e-09,5.984481e-09,5.367942e-09,5.670207e-09,4.698269e-09,7.574770e-09,7.410458e-09,5.331319e-09,...,8.147294e-09,6.749451e-09,7.545797e-09,6.758675e-09,8.548588e-09,8.766245e-09,7.990896e-09,8.931241e-09,6.439181e-09,0
1997,4.990997e-09,4.787734e-09,5.932627e-09,6.827147e-09,5.931501e-09,8.075331e-09,8.788223e-09,8.800377e-09,7.806931e-09,7.991977e-09,...,8.863889e-09,1.004460e-08,9.469247e-09,9.847104e-09,9.357064e-09,8.641800e-09,9.444979e-09,9.498395e-09,7.682136e-09,0
1998,6.218237e-09,7.594114e-09,6.446069e-09,7.649783e-09,8.353429e-09,8.974337e-09,8.134333e-09,7.239301e-09,8.452860e-09,8.481620e-09,...,8.550503e-09,9.066814e-09,7.707216e-09,8.250646e-09,6.913003e-09,9.555235e-09,1.001332e-08,9.327360e-09,9.027125e-09,0


In [90]:
transposed_df.to_csv("wide184.csv", index=False)